In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
%matplotlib inline

In [ ]:
# Set the image size.
img_height = 300
img_width = 300

In [ ]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = 'path/to/trained/model.h5'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'DecodeDetections': DecodeDetections,
                                               'compute_loss': ssd_loss.compute_loss})

In [ ]:
import numpy as np
import cv2
frames=np.zeros((1,300,300,3))
cap = cv2.VideoCapture(0)
confidence_threshold = 0.3
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'umbrella']
font                   = cv2.FONT_HERSHEY_SIMPLEX

fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    x = 20
    y = 40
    w = 100
    h = 75
    frames[0] = image.load_img(img_path, target_size=(img_height, img_width))
    y_pred = model.predict(frames)
    
    
    np.set_printoptions(precision=2, suppress=True, linewidth=90)
    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
    for box in y_pred_thresh[0]:
        xmin = box[2] 
        ymin = box[3] 
        xmax = box[4] 
        ymax = box[5] 
        color = colors[int(box[0])]
        cv2.putText(frame,classes[int(box[0])], 
        (xmin,ymin), 
        font, 
        fontScale,
        fontColor,
        lineType)

        cv2.rectangle(frame, (xmin, xmin), (xmax, ymax), color, 2)    
    # Our operations on the frame come here
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    else:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        

    # Display the resulting frame
    cv2.imshow('frame',frame)
    

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()